In [15]:
import os
from langchain_openai.chat_models.azure import AzureChatOpenAI

openai_api_key = os.getenv('AZURE_OPENAI_API_KEY')
openai_api_base = os.getenv('AZURE_OPENAI_ENDPOINT')
openai_api_version = os.getenv('AZURE_OPENAI_API_VERSION')
embeddings_model = os.getenv('AZURE_OPENAI_EMBEDDING_DEPLOYMENT_NAME')
gpt_model = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME")

llm = AzureChatOpenAI(
                openai_api_version=openai_api_version,
                azure_deployment=gpt_model,
            )

In [57]:
from langchain_core.prompts import PromptTemplate

def diagram_agent(text):
    "you are a diagram creator expert that generates diagrams with mermaid package"

    prompt = """Your job is to write the code to generate a colorful mermaid diagram describing the text below:

    {text} 
    
    Please understand the text before generating the code. use the text information only , don't use any other information.
    only generate the code as output nothing extra.
    each line in the code must be terminated by ; 
    Code:"""

    template = PromptTemplate.from_template(prompt)
    chain = template | llm
    response = chain.invoke({"text": text})
    return response.content
    

In [58]:
from mermaid import Mermaid

def execute_mermaid(graph):

    if "mermaid" in graph:
        graph = graph.replace("mermaid","").replace("`", "")
    return Mermaid(graph)


In [60]:
import sys
import os
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_community.vectorstores import FAISS
from langchain_openai import AzureOpenAIEmbeddings

#sys.path.append(os.path.abspath("../api/"))

embeddings = AzureOpenAIEmbeddings(
                azure_endpoint=os.getenv('AZURE_OPENAI_ENDPOINT'),
                api_key=os.getenv('AZURE_OPENAI_API_KEY'),
                azure_deployment=os.getenv('AZURE_OPENAI_EMBEDDING_DEPLOYMENT_NAME')
            )


index_path =  "../api/index/faiss_index"
vector_store = FAISS.load_local(index_path, embeddings, allow_dangerous_deserialization=True)
retriever = vector_store.as_retriever()
template = """Answer questions based only on the following context: 
        
        ### CONTEXT ###
        {context}
        
        Question: {question}
        
        If the context doesn't provide enough information to answer the question, say that you don't know.
        Do not respond with information outside the previous context."""

prompt = PromptTemplate.from_template(template)

chain = (
            {"context": retriever, "question": RunnablePassthrough()}
            | prompt
            | llm
            | StrOutputParser()
        )

query="what python libraries can I use for art generation"
result = chain.invoke(query)

print(result)

You can use the following Python libraries for art generation:

1. **Matplotlib** - A plotting library used for creating static, animated, and interactive visualizations.
2. **NumPy** - A library that adds support for large multi-dimensional arrays and matrices, providing high-level mathematical functions for operations on these arrays.
3. **Turtle** - A pre-installed library that allows users to create various types of shapes and images on a virtual canvas.
4. **Pygame** - While not explicitly mentioned in the provided context, it is noted as one of the libraries that can be used for generative artwork.
5. **Processing.py** - Another library referenced for its capabilities in creating visuals, though not elaborated upon in the context.
6. **TensorFlow** - Used for implementing generative adversarial networks (GANs) which can create realistic images.

These libraries facilitate the creation of generative artwork by enabling various types of visualizations, manipulations, and interactio

In [47]:
graph = diagram_agent(text = result)

In [53]:
print(graph)

```mermaid
graph TD;
    A[Install Python and required libraries] --> B[Define the generative art piece];
    B --> C[Import libraries];
    C --> D[Create a blank canvas];
    D --> E[Generate the art];
    E --> F[Display the generated art];
    
    style A fill:#ffcccb,stroke:#ff0000,stroke-width:2px;
    style B fill:#add8e6,stroke:#0000ff,stroke-width:2px;
    style C fill:#90ee90,stroke:#008000,stroke-width:2px;
    style D fill:#ffffe0,stroke:#ffff00,stroke-width:2px;
    style E fill:#d8bfd8,stroke:#9370db,stroke-width:2px;
    style F fill:#ffe4e1,stroke:#ff69b4,stroke-width:2px;
```


In [59]:
execute_mermaid(graph)